<a href="https://colab.research.google.com/github/vfrantc/weather_experiments/blob/main/run_refinednet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Jul 14 08:42:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!git clone https://github.com/xiaofeng94/RefineDNet-for-dehazing.git

Cloning into 'RefineDNet-for-dehazing'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 131 (delta 54), reused 108 (delta 39), pack-reused 0
Receiving objects: 100% (131/131), 2.41 MiB | 9.78 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [4]:
%cd RefineDNet-for-dehazing

/content/RefineDNet-for-dehazing


In [5]:
!cp /content/drive/MyDrive/weather_experiments/weather_test.zip .
!unzip weather_test.zip 

Streaming output truncated to the last 5000 lines.
  inflating: test/outdoor/input/0194_1_0.2.jpg  
  inflating: test/outdoor/input/1986_0.9_0.08.jpg  
  inflating: test/outdoor/input/0251_0.8_0.16.jpg  
  inflating: test/outdoor/input/1953_0.8_0.2.jpg  
  inflating: test/outdoor/input/0127_0.8_0.2.jpg  
  inflating: test/outdoor/input/0003_0.8_0.2.jpg  
  inflating: test/outdoor/input/1861_0.85_0.2.jpg  
  inflating: test/outdoor/input/0317_0.85_0.08.jpg  
  inflating: test/outdoor/input/0269_0.95_0.2.jpg  
  inflating: test/outdoor/input/1055_0.95_0.2.jpg  
  inflating: test/outdoor/input/0267_0.9_0.16.jpg  
  inflating: test/outdoor/input/1848_0.8_0.12.jpg  
  inflating: test/outdoor/input/0082_0.85_0.16.jpg  
  inflating: test/outdoor/input/0138_0.9_0.08.jpg  
  inflating: test/outdoor/input/1724_0.95_0.2.jpg  
  inflating: test/outdoor/input/1889_0.85_0.08.jpg  
  inflating: test/outdoor/input/0302_0.9_0.16.jpg  
  inflating: test/outdoor/input/1868_1_0.08.jpg  
  inflating: test/

In [6]:
!cp /content/drive/MyDrive/refined_DCP_outdoor.zip .
!unzip refined_DCP_outdoor.zip

Archive:  refined_DCP_outdoor.zip
   creating: refined_DCP_outdoor/
  inflating: refined_DCP_outdoor/test_opt.txt  
  inflating: refined_DCP_outdoor/60_net_Refiner_T.pth  
  inflating: refined_DCP_outdoor/60_net_Refiner_J.pth  
  inflating: refined_DCP_outdoor/60_net_D.pth  


In [7]:
!ls test

fog	     rain	     raindroptesta.txt	sand	    snowtest100k_L.txt
fog.txt      rain1400	     rain.txt		sand.txt    snow.txt
outdoor      rain1400.txt    real		snow	    test1
outdoor.txt  rain_drop_test  real.txt		Snow100K-L  test1.txt


In [9]:
!mkdir -p datasets/fog
!cp -r test/fog/input datasets/fog/testA
!mkdir -p datasets/outdoor
!cp -r test/outdoor/input datasets/outdoor/testA
!mkdir -p datasets/real 
!cp -r test/real/input datasets/real/testA
!mkdir -p datasets/sand
!cp -r test/sand/input datasets/sand/testA
!mkdir -p datasets/snow
!cp -r test/snow/input datasets/snow/testA
!mkdir -p datasets/test1
!cp -r test/test1/input datasets/test1/testA

In [10]:
!mkdir checkpoints
!mv refined_DCP_outdoor checkpoints

In [11]:
import os
import cv2
from glob import glob
from tqdm.notebook import tqdm

In [12]:
def presize(folder):
  sizes = dict()
  for imfile in tqdm(list(glob(os.path.join(folder, '*.png'))) + list(glob(os.path.join(folder, '*.jpg')))):
    image = cv2.imread(imfile)
    (height, width) = image.shape[:2]
    key = os.path.basename(imfile)
    sizes[key] = (height, width)

    height, width = image.shape[:2]
    msize = min(height, width)
    if msize < 256:
      if height < width:
        ratio = width / height
        new_height = 256
        new_width = int(new_height*ratio)
      else:
        ratio = height / width
        new_width = 256
        new_height = int(new_width*ratio)
      image = cv2.resize(image, (new_width, new_height))
      cv2.imwrite(imfile, image)

In [13]:
presize('datasets/fog/testA')
presize('datasets/outdoor/testA')
presize('datasets/real/testA')
presize('datasets/sand/testA')
presize('datasets/snow/testA')
presize('datasets/test1/testA')

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/497 [00:00<?, ?it/s]

  0%|          | 0/323 [00:00<?, ?it/s]

  0%|          | 0/204 [00:00<?, ?it/s]

  0%|          | 0/750 [00:00<?, ?it/s]

In [14]:
!python3 quick_test.py --dataroot ./datasets/fog --dataset_mode single --name refined_DCP_outdoor --model refined_DCP --phase test --preprocess none --save_image --method_name refined_DCP_outdoor_ep_60 --epoch 60
!python3 quick_test.py --dataroot ./datasets/outdoor --dataset_mode single --name refined_DCP_outdoor --model refined_DCP --phase test --preprocess none --save_image --method_name refined_DCP_outdoor_ep_60 --epoch 60
!python3 quick_test.py --dataroot ./datasets/real --dataset_mode single --name refined_DCP_outdoor --model refined_DCP --phase test --preprocess none --save_image --method_name refined_DCP_outdoor_ep_60 --epoch 60
!python3 quick_test.py --dataroot ./datasets/sand --dataset_mode single --name refined_DCP_outdoor --model refined_DCP --phase test --preprocess none --save_image --method_name refined_DCP_outdoor_ep_60 --epoch 60
!python3 quick_test.py --dataroot ./datasets/snow --dataset_mode single --name refined_DCP_outdoor --model refined_DCP --phase test --preprocess none --save_image --method_name refined_DCP_outdoor_ep_60 --epoch 60
!python3 quick_test.py --dataroot ./datasets/test1 --dataset_mode single --name refined_DCP_outdoor --model refined_DCP --phase test --preprocess none --save_image --method_name refined_DCP_outdoor_ep_60 --epoch 60

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./datasets/fog                	[default: ./datasets/ITS_v2]
             dataset_mode: single                        	[default: unaligned]
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: 60                            	[default: latest]
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
    

In [25]:
!mkdir refinednet
!cp -r datasets/fog/refined_DCP_outdoor_ep_60 refinednet/fog
!cp -r datasets/outdoor/refined_DCP_outdoor_ep_60 refinednet/outdoor
!cp -r datasets/real/refined_DCP_outdoor_ep_60 refinednet/real
!cp -r datasets/sand/refined_DCP_outdoor_ep_60 refinednet/sand
!cp -r datasets/snow/refined_DCP_outdoor_ep_60 refinednet/snow
!cp -r datasets/test1/refined_DCP_outdoor_ep_60 refinednet/test1

mkdir: cannot create directory ‘refinednet’: File exists


In [26]:
import shutil

In [27]:
def ren(name):
  files = list(glob(os.path.join(name, '*.jpg'))) + list(glob(os.path.join(name, '*.png')))
  for file in tqdm(files):
      shutil.move(file, file.replace('_dehz', ''))

In [28]:
ren('refinednet/fog')
ren('refinednet/outdoor')
ren('refinednet/real')
ren('refinednet/sand')
ren('refinednet/snow')
ren('refinednet/test1')

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/497 [00:00<?, ?it/s]

  0%|          | 0/323 [00:00<?, ?it/s]

  0%|          | 0/204 [00:00<?, ?it/s]

  0%|          | 0/750 [00:00<?, ?it/s]

In [29]:
!zip -r refinednet.zip refinednet/
!cp refinednet.zip /content/drive/MyDrive/weather_experiments/

  adding: refinednet/ (stored 0%)
  adding: refinednet/real/ (stored 0%)
  adding: refinednet/real/195.png (deflated 0%)
  adding: refinednet/real/663.png (deflated 0%)
  adding: refinednet/real/169.png (deflated 0%)
  adding: refinednet/real/367.png (deflated 0%)
  adding: refinednet/real/260.png (deflated 0%)
  adding: refinednet/real/559.png (deflated 1%)
  adding: refinednet/real/271.png (deflated 0%)
  adding: refinednet/real/003.png (deflated 0%)
  adding: refinednet/real/008.png (deflated 0%)
  adding: refinednet/real/113.png (deflated 0%)
  adding: refinednet/real/135.png (deflated 0%)
  adding: refinednet/real/631.png (deflated 0%)
  adding: refinednet/real/290.png (deflated 0%)
  adding: refinednet/real/257.png (deflated 1%)
  adding: refinednet/real/469.png (deflated 0%)
  adding: refinednet/real/350.png (deflated 0%)
  adding: refinednet/real/495.png (deflated 0%)
  adding: refinednet/real/214.png (deflated 1%)
  adding: refinednet/real/591.png (deflated 1%)
  adding: refin